<a href="https://colab.research.google.com/github/goransavich/cnn-xgboost/blob/main/cnn_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot
np.random.seed(1337)  # for reproducibility
#from keras.utils import np_utils
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Convolution1D, MaxPooling1D, Flatten, Input, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
#xgboost
import xgboost as xgb
import pandas as pd
from sklearn import metrics

In [ ]:
from google.colab import files
uploaded = files.upload()
import io
import pandas as pd

train_name = list(uploaded.keys())[0]

train_df = pd.read_csv(io.BytesIO(uploaded[train_name]))

Saving Coffee_TRAIN.csv to Coffee_TRAIN (1).csv


In [ ]:
uploaded = files.upload()

test_name = list(uploaded.keys())[0]

test_df = pd.read_csv(io.BytesIO(uploaded[test_name]))

Saving Coffee_TEST.csv to Coffee_TEST (1).csv


In [ ]:
train_data = np.array(train_df)
test_data = np.array(test_df)

In [ ]:
x_train = train_data[:, :-1]
x_test = test_data[:, :-1]
train_y = train_data[:, -1:] ###OVDE VIDETI KAKO DA NAPRAVIM Y TEST DA IMA 22 KOMADA
test_y = test_data[:, -1:]

classes_in_dataset = len(np.unique(np.concatenate((train_y, test_y), axis=0)))

# transform the labels from integers to one hot vectors
enc = OneHotEncoder(categories='auto')
enc.fit(np.concatenate((train_y, test_y), axis=0).reshape(-1, 1))
y_train = enc.transform(train_y.reshape(-1, 1)).toarray()
y_test = enc.transform(test_y.reshape(-1, 1)).toarray()


In [ ]:
x_train,x_validate,y_train,y_validate = train_test_split(x_train,y_train,test_size = 0.2,random_state = 42)

In [ ]:
if len(x_train.shape) == 2:  # if univariate
        # add a dimension to make it multivariate with one dimension 
        x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
        x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
input_shape = x_train.shape[1:]

print(input_shape)

print(classes_in_dataset)
###y_train for xgboost without encoding
y_train_xgboost = enc.inverse_transform(y_train)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
print(x_validate.shape)
print(y_validate.shape)

(286, 1)
2
(22, 286, 1)
(22, 2)
(28, 286, 1)
(28, 2)
(6, 286)
(6, 2)


In [ ]:
inputs = input_shape
input_layer = Input(inputs)
conv2d1 = Convolution1D(filters=64, kernel_size=1, padding='valid')(input_layer)
activation1 = Activation('relu')(conv2d1)
maxpooling1 = MaxPooling1D(pool_size=2, strides=1, padding='valid')(activation1)
#dropout1 = Dropout(0.5)(maxpooling1)

conv2d2 = Convolution1D(filters=62, kernel_size=1, padding='valid')(maxpooling1)
activation2 = Activation('relu')(conv2d2)
maxpooling2 = MaxPooling1D(pool_size=2, strides=1, padding='valid')(activation2)
dropout2 = Dropout(0.5)(maxpooling2)

flatten = Flatten()(maxpooling2)

dense1 = Dense(256)(flatten)
activation3 = Activation('relu')(dense1)

dense2 = Dense(units=classes_in_dataset)(activation3)
activation4 = Activation('softmax')(dense2)

output_for_xgboost = Model(inputs=input_layer, outputs=activation3)
model = Model(inputs=input_layer, outputs=activation4)

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model.compile(loss ='categorical_crossentropy', optimizer=Adam(learning_rate=0.001),metrics =['accuracy'])

history = model.fit(
    x_train,
    y_train,
    batch_size=2,
    epochs=75,
    verbose=1,
    callbacks=[callback],
    validation_data=(x_validate,y_validate),
)

score = model.evaluate(x_test,y_test,verbose=0)
print('Test Loss : {:.4f}'.format(score[0]))
print('Test Accuracy : {:.4f}'.format(score[1]))


######### XGBOOST #############


intermediate_layer_model = output_for_xgboost

intermediate_layer_model.summary()





x_train_xgboost = intermediate_layer_model.predict(x_train)
x_test_xgboost = intermediate_layer_model.predict(x_test)



xgboost_train_dataset = np.concatenate([np.array(x_train_xgboost).reshape(x_train.shape[0],256),np.array(y_train_xgboost).reshape(y_train_xgboost.shape[0],1)],axis=1)
xgboost_train_dataset = pd.DataFrame(xgboost_train_dataset)
xgboost_train_dataset.to_csv('train_xgboost.csv',index=False)

xgboost_test_dataset = np.concatenate([np.array(x_test_xgboost).reshape(x_test.shape[0],256),np.array(test_y).reshape(test_y.shape[0],1)],axis=1)
xgboost_test_dataset = pd.DataFrame(xgboost_test_dataset)
xgboost_test_dataset.to_csv('test_xgboost.csv',index=False)

train = pd.read_csv('train_xgboost.csv')
train_y = train['256'].astype('int')
train_x = train.drop(['256'],axis=1)

#print(x_train_xg.shape)
#print(x_test_xg.shape)

test = pd.read_csv('test_xgboost.csv')
test_y = test['256'].astype('int')
test_x = test.drop(['256'],axis=1)

dataset = xgb.DMatrix(train_x, label=train_y)
watchlist = [(dataset, 'train')]
params = {'max_depth':4, 'eta':0.3, 'silent':1, 'num_class': classes_in_dataset} 
model_xg = xgb.train(params, dataset, num_boost_round=50, evals='watchlist')


test_x = xgb.DMatrix(test_x)
result = model_xg.predict(test_x)


Epoch 1/75
11/11 [==============================] - 1s 91ms/step - loss: 1.2410 - accuracy: 0.5455 - val_loss: 0.6441 - val_accuracy: 0.6667
Epoch 2/75
11/11 [==============================] - 1s 78ms/step - loss: 0.8017 - accuracy: 0.4545 - val_loss: 0.6568 - val_accuracy: 0.6667
Epoch 3/75
11/11 [==============================] - 1s 76ms/step - loss: 0.6920 - accuracy: 0.4545 - val_loss: 0.6988 - val_accuracy: 0.3333
Epoch 4/75
11/11 [==============================] - 1s 75ms/step - loss: 0.6870 - accuracy: 0.5455 - val_loss: 0.7803 - val_accuracy: 0.3333
Epoch 5/75
11/11 [==============================] - 1s 76ms/step - loss: 0.6790 - accuracy: 0.5455 - val_loss: 0.7320 - val_accuracy: 0.3333
Epoch 6/75
11/11 [==============================] - 1s 74ms/step - loss: 0.6901 - accuracy: 0.6364 - val_loss: 0.6615 - val_accuracy: 1.0000
Epoch 7/75
11/11 [==============================] - 1s 76ms/step - loss: 0.6724 - accuracy: 0.8182 - val_loss: 0.6932 - val_accuracy: 0.3333
Epoch 8/75
11

TypeError: ignored

In [ ]:

from sklearn.metrics import precision_score, recall_score, accuracy_score

preds = model.predict(d_test)
best_preds = np.asarray([np.argmax(line) for line in result])

print("Precision = {}".format(precision_score(test_y, best_preds, average='macro')))
print("Recall = {}".format(recall_score(test_y, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(test_y, best_preds)))